In [ ]:
#نادين اشرف كمال مغربي
#هبة الله محمد ممدوح
#ريم وليد علي الشناوي
#مصطفي عادل ذكريا حسن
#مريم ايمن محمد السيد

In [ ]:
import numpy as np
import pandas as pd
import itertools
from itertools import permutations
import io
from google.colab import files
from numpy.core.fromnumeric import diagonal
uploaded = files.upload()
BlosumMatrix = pd.read_csv(io.BytesIO(uploaded['blosum62.csv']))
print(BlosumMatrix)

In [ ]:
def BlosumDictionary(matrix):
    AminoAcids_Scores=BlosumMatrix.values.tolist() #convert to list that contains 24 lists
    charVecs = []#empty list
    for ch in range(0,2):
        charVecs.append(list(matrix.index))
    AminoAcidLetters1 = charVecs[0] #first letters in csv file
    AminoAcidLetters2 = charVecs[1] #second letters in csv file
    AminoAcidCombined= list(itertools.product(AminoAcidLetters1,AminoAcidLetters2))
    K = 24 #rows in csv file
    AminoAcidKeys= []
    for idx in range(0, K):#to make the amino acid list of list
        AminoAcidKeys.append( AminoAcidCombined[idx::K]) #reverse combination
    BlosumDictionary=dict(zip(AminoAcidKeys[0],AminoAcids_Scores[0]))
    for i in range(1,24):
        IteratedDictionary= dict(zip(AminoAcidKeys[i],AminoAcids_Scores[i]))

        BlosumDictionary.update(IteratedDictionary)
    return BlosumDictionary

In [ ]:
blosumScore=BlosumDictionary(BlosumMatrix)

In [ ]:
def split(string):
  return list(string)

In [ ]:
def tracebackDNA(rowNum,colNum,Seq1,Seq2,Matrix):
  row=rowNum
  column=colNum
  LAFirst =""
  LASecond=""
  LAMatch=""
  while row > 0 and column > 0:
      up = Matrix[row-1][column]-1
      if up<0: up=0
      left = Matrix[row][column-1]-1
      if left<0: left=0
      if Seq1[column-1]==Seq2[row-1]:
        score=1
      else:
        score=-1
      diagonal = Matrix[row-1][column-1]+score
      if diagonal<0: diagonal=0
      if Matrix[row][column]==diagonal:
        LAFirst+=Seq1[column-1]  #adds letters from first sequence
        LASecond+=Seq2[row-1] #adds letters from second sequence
        if score==1:
          LAMatch+="|"
        else:
          LAMatch+=" "
        row-=1 #leave row
        column-=1 #leave column
      elif Matrix[row][column] == up:
        LAFirst+="-"
        LAMatch+=" "
        LASecond+=Seq2[row-1]
        row-=1 #leave row
      else:
        LAFirst+=Seq1[column-1]
        LASecond+="-"
        LAMatch+=" "
        column-=1 #leave column
  while (row>0):
        LAFirst+="-"
        LASecond+=Seq2[row-1]
        LAMatch+=" "
        row-=1
  while(column>0):
        LAFirst+=Seq1[column-1]
        LASecond+="-"
        LAMatch+=" "
        column-=1

  LAFirst=LAFirst[::-1] #prints letters in correct order (reversed)
  LAMatch=LAMatch[::-1]
  LASecond=LASecond[::-1]
  print("Traceback of this DNA sequence:")
  print(LAFirst)
  print(LAMatch)
  print(LASecond)


In [ ]:
def tracebackProtein(rowEnd,columnEnd,Seq1,Seq2,Matrix):
    LAFirst =""
    LASecond=""
    row=rowEnd
    column=columnEnd
    if row > column:
              MinLength = column
    else:
              MinLength = row
    splitSeq1= split(Seq1)
    splitSeq2= split(Seq2)
    while row > 0 and column > 0:
          up = Matrix[row-1][column]-1
          if up<0:
            up=0
          left=Matrix[row][column-1]-1
          if left<0:
            left =0
          for i in range(MinLength):
            diagonal = Matrix[row-1][column-1]+blosumScore[(splitSeq1[column-1],splitSeq2[row-1])]
          if diagonal<0:
            diagonal=0
          if Matrix[row][column]==diagonal:
            LAFirst+=Seq1[column-1]
            LASecond+=Seq2[row-1]
            row-=1
            column-=1
          elif Matrix[row][column] == up:
            LAFirst+="-"
            LASecond+=Seq2[row-1]
            row-=1
          else:
            LAFirst+=Seq1[column-1]
            LASecond+="-"
            column-=1
    while (row>0):
        LAFirst+="-"
        LASecond+=Seq2[row-1]
        row-=1
    while(column>0):
        LAFirst+=Seq1[column-1]
        LASecond+="-"
        column-=1

    LAFirst=LAFirst[::-1]
    LASecond=LASecond[::-1]
    print("Traceback of this protein sequence:")
    print(LAFirst)
    print(LASecond)

In [ ]:
def localAlignmentDNA(Seq1,Seq2):
        lengthOfFirstSeq=len(Seq1)  #what represents columns
        lengthOfSecondSeq=len(Seq2) #what represents rows
        Matrix = [[0 for row in range(lengthOfFirstSeq+1)] for columns in range(lengthOfSecondSeq+1)]
        print("Shape of Matrix is : "+str(np.shape(Matrix)))
        for row in range (lengthOfSecondSeq+1):
            Matrix[row][0]=0 #fixed column and adding gaps to every row in that column
        for columns in range (lengthOfFirstSeq+1):
            Matrix[0][columns]=0  #fixed row and adding gaps to every column in that row
        for row in range(1,lengthOfSecondSeq+1,1):
            for column in range(1,lengthOfFirstSeq+1,1):
                if Seq1[column -1] == Seq2[row-1]: #to see if a letter matches the other and sets the score
                    score = 1
                else :
                    score =-1
                Matrix[row][column] = max(0,Matrix[row-1][column]-1, Matrix[row][column-1]-1,Matrix[row-1][column-1]+score)
                #
        for row in range (lengthOfSecondSeq+1):
            for col in range (lengthOfFirstSeq+1):
                print(Matrix[row][col],end="")
            print("\n")
        list=[]
        dict={}
        for row in range(lengthOfSecondSeq+1):
            a_row = Matrix[row] # taking each row in a variable
            max_value = max(a_row) # find max value in a single row
            colindex = a_row.index(max_value) # find the max value's index of a single row
            list.append(max_value)
            dict[row]=colindex # has row(as key) and column index(as value) of every max in each row
            print("The max value ",max_value, " and Index in ",colindex,"The row",row)
        print(list)
        print(dict)
        max_value = max(list)
        max_index = list.index(max_value)
        row=max_index #assigns index of max to "row"
        col=dict[max_index] #assigns key of max to "col"
        print("Row where max: ",row)
        print("Col where max: ",col)
        score= Matrix[row][col]
        print("Score: " ,score)
        tracebackDNA(row,col,Seq1,Seq2,Matrix)


In [ ]:
def localAlignmentProtein(Seq1,Seq2):
  lengthOfFirstSeq=len(Seq1) #what represents columns
  lengthOfSecondSeq=len(Seq2) #what represents rows
  Matrix = [[0 for row in range(lengthOfFirstSeq+1)] for columns in range(lengthOfSecondSeq+1)]
  print("Shape of Matrix is : "+str(np.shape(Matrix)))
  for row in range (lengthOfSecondSeq+1):
        Matrix[row][0]=0 #fixed column and adding gaps to every row in that column
  for columns in range (lengthOfFirstSeq+1):
        Matrix[0][columns]=0  #fixed row and adding gaps to every column in that row
  splitSeq1= split(Seq1)
  splitSeq2= split(Seq2)
  for row in range(1,lengthOfSecondSeq+1,1):
      for column in range(1,lengthOfFirstSeq+1,1):
          Matrix[row][column] = max(0,Matrix[row-1][column]-1, Matrix[row][column-1]-1,Matrix[row-1][column-1]+blosumScore[(splitSeq1[column-1],splitSeq2[row-1])])
  for row in range (lengthOfSecondSeq+1):
            for col in range (lengthOfFirstSeq+1):
                print(Matrix[row][col],end="")
            print("\n")
  list=[]
  dict={}
  for i in range(lengthOfSecondSeq+1):
            a_row = Matrix[i] # taking each row in a variable
            max_value = max(a_row) # find max value in a single row
            colindex = a_row.index(max_value) # find the max value's index of a single row
            list.append(max_value)
            dict[i]=colindex # has row(as key) and column index(as value) of every max in each row
            print("The max value ",max_value, " and Index in ",colindex,"The row",i)
  print(list)
  print(dict)
  max_value = max(list)
  max_index = list.index(max_value)
  rowEnd=max_index #assigns index of max to "row"
  colEnd=dict[max_index] #assigns key of max to "col"
  print("Row where max: ",rowEnd)
  print("Col where max: ",colEnd)
  score= Matrix[rowEnd][colEnd]
  print("Score: " ,score)
  tracebackProtein(rowEnd,colEnd,Seq1,Seq2,Matrix) #sends max in matrix to start from at traceback



In [ ]:
def correctFormat(Seq1,Seq2,seqCheck):
  correct1=all(characters in seqCheck for characters in Seq1)
  correct2=all(characters in seqCheck for characters in Seq2)
  return correct1 and correct2 #sends true if both are correct

In [ ]:
def checkSequence(typeSeq):
  if typeSeq=="1":
    Seq1=input("Enter First Sequence ").upper()
    Seq2=input("Enter Second Sequence ").upper()
    DNAseq='ACTG'
    correct=correctFormat(Seq1,Seq2,DNAseq)
    if(correct==True):
      localAlignmentDNA(Seq1,Seq2)
    else:
      print("Invalid sequence")
      checkSequence(typeSeq)

  elif typeSeq=="2":
    Seq1=input("Enter First Sequence ").upper()
    Seq2=input("Enter Second Sequence ").upper()
    proteinSeq='ACDBEFGHIKLMNPQRSTVWY'
    correct=correctFormat(Seq1,Seq2,proteinSeq) #send the sequences and which sequence format it should have (DNA or Protein Letters)
    if(correct==True):
        localAlignmentProtein(Seq1,Seq2)
    else:
        print('Invalid sequence')
        checkSequence(typeSeq)

In [ ]:
print("Local Alignment")
typeSeq = input("1-DNA , 2-Amino Acid ")
checkSequence(typeSeq)